## **<center>Case Study 4</center>**

**<center>Author: Yi Zhang</center>**

**<center>Uni: yz4130</center>**

In [3]:
import numpy as np
from scipy.optimize import fmin

#### **Simulation of Asset Price**

In [4]:
# True parameters
true_params = [0.02, 1.5, 0.05, 0.18, 0.5, 0.04]

In [5]:
# Euler scheme to simulate stock price under Heston dynamic
def heston_euler(params, T, y0):
    mu, kappa, theta, lam, rho, v0 = params
    dt = 1/T
    sim = np.zeros((T+1, 2))
    sim[0,:] = y0, v0
    z = np.random.normal(size=(T,2))
    for i in range(1, T+1):
        y_last, v_last = sim[i-1,:]
        
        # use full truncation to avoid negative vol values
        vk = v_last + kappa * (theta - max(v_last, 0)) * dt + lam * np.sqrt(max(v_last, 0) * dt) * z[i-1, 0]
        
        z_correlated = rho * z[i-1, 0] + np.sqrt(1-rho**2) * z[i-1, 1]
        yk = y_last + (mu - 0.5*v_last) * dt + np.sqrt(v_last*dt) * z_correlated
        
        sim[i,:] = yk, vk
    return sim[:,0]

#### **Extended Kalman Filter**

In [4]:
# given price history, calculate the log likelihood
def extKalman(init, T, H, P, logprice):
    mu, kappa, theta, lam, rho, v0 = init
    dt = 1/T
    Q = np.array([[1, rho],
                  [rho, 1]])

    F = np.array([[1, -0.5*dt],   
                  [0, 1-kappa*dt]])
    
    v = v0
    likelihood = 0
    
    for t in range(1, T+1):
        U = np.diag([np.sqrt(v*dt), lam * np.sqrt(v*dt)])
        y_last = logprice[t-1]
        
        # time update
        x1 = np.array([y_last + (mu - 0.5 * v) * dt, 
                       v + kappa * (theta - v) * dt])
        P1 = F @ P @ F.T + U @ Q @ U.T
        S = H @ P1 @ H.T        
        K = P1 @ H.T / S
        v1 = x1[1]
        
        # calculate error and update likelihood cost function
        e = (logprice[t] - y_last) - (mu - 0.5*v1) * dt
        likelihood += np.log(S) + e**2/S
        
        # measurement update
        x = x1 + K * e
        v = x[1]
        P = (np.diag([1, 1]) - np.outer(K, H)) @ P1
 
    return likelihood

In [68]:
H = np.array([1, 0])
P = np.diag([0.01, 0.01])
init = [0.033, 1.7, 0.08, 0.14, 0.36, 0.023]

T = 10 * 365
logprice0 = 3.66
heston_price = heston_euler(true_params, T, logprice0)

extended_model = fmin(extKalman, init, args=(T, H, P, heston_price), full_output=True, disp=True, xtol=10, ftol=20)
extKalman_params = np.round(extended_model[0], 4)

Optimization terminated successfully.
         Current function value: -39986.580913
         Iterations: 28
         Function evaluations: 48


In [69]:
print('Estimated parameters:', extKalman_params)
print('True parameter:', true_params)
print('Maximum Absolute Error:', np.round(np.max(extKalman_params - true_params), 4))
print('Mean Square Error:', np.round(np.linalg.norm(extKalman_params - true_params), 4))


Estimated parameters: [0.0313 1.6764 0.0221 0.1863 0.4315 0.0251]
True parameter: [0.02, 1.5, 0.05, 0.18, 0.5, 0.04]
Maximum Absolute Error: 0.1764
Mean Square Error: 0.1923


#### **Particle Filter**

In [10]:
def normal_pdf(x, m, s):
    expon = -0.5 * ((x - m) / s) ** 2
    deno = np.sqrt(2 * np.pi) * s
    return np.exp(expon) / deno


# resampling to method to avoid degeneracy
def resampling(w, x):
    nSim = len(x)
    x_new = np.zeros(nSim)
    cum_sum = np.cumsum(w)
    i = 0
    for j in range(N):
        uj = (np.random.uniform() + j) / nSim
        while uj > cum_sum[i]:
            i += 1
        x_new[j] = x[i]
    return x_new, np.ones(nSim)/nSim  


def parFilter(init, T, logprice, N):
    mu, kappa, theta, lam, rho, v0 = init
    dt = 1/T
    weight = np.ones(N) / N
    const = (1 + 0.5 * rho * lam * dt) ** (-1)
    
    v = v0 * np.ones(N)
    likelihood = 0 
    
    for t in range(1, T+1):        
        yt, y_last = logprice[t], logprice[t-1]
        
        # draw N samples from prior
        term1 = v + kappa * (theta - v) * dt
        term2 = lam * rho * 0.5 * v * dt
        term3 = lam * np.sqrt(v * (1 - rho**2) * dt) * np.random.normal(size=N)
        term4 = lam * rho * np.sqrt(v * dt) * np.random.normal(size=N)
        
        # equation 11 from Lecture 12
        v_next = (term1 + term2 + term3 + term4) * const

        # transition density
        m_T = const * (v + kappa * (theta - v) * dt + 0.5 * lam * rho * v * dt)
        sig_T = const * lam * np.sqrt(v*dt)
        
        # importance function
        e = (yt - y_last) - (mu - 0.5*v) * dt
        m_i = v + kappa * (theta - v) * dt + lam * rho * e
        sig_i = lam * np.sqrt(v * (1 - rho**2) * dt)
        
        # likelihood
        m_L = y_last + (mu - 0.5*v_next) * dt
        sig_L = np.sqrt(v * dt)
        
        # update cost function
        ratio = normal_pdf(yt, m_L, sig_L) * normal_pdf(v_next, m_T, sig_T) / normal_pdf(v_next, m_i, sig_i)
        weight = ratio * weight
        likelihood += np.log(np.sum(weight))
        
        # normalize the weight and do resampling
        weight = weight / np.sum(weight)  
        v, weight = resampling(weight, v_next)
        
    return -likelihood
    

In [20]:
init = [0.024, 1.79, 0.088, 0.14, 0.36, 0.053]
T = 10 * 365
logprice0 = 3.66
heston_price = heston_euler(true_params, T, logprice0)
# number of particles
N = 20

particle_model = fmin(parFilter, init, args=(T, heston_price, N), full_output=True, disp=True, xtol=10, ftol=50)
particle_params = np.round(particle_model[0], 4)

/Users/Yi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: RuntimeWarning: divide by zero encountered in true_divide
/Users/Yi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:61: RuntimeWarning: invalid value encountered in true_divide
/Users/Yi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: RuntimeWarning: invalid value encountered in true_divide
/Users/Yi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in log
/Users/Yi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in sqrt
/Users/Yi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in sqrt
/Users/Yi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in sqrt
/Users/Yi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: Runti

In [21]:
print('Estimated parameters:', particle_params)
print('True parameter:', true_params)
print('Maximum Absolute Error:', np.round(np.max(particle_params - true_params), 4))
print('Mean Square Error:', np.round(np.linalg.norm(particle_params - true_params), 4))

Estimated parameters: [0.0245 2.0705 0.0331 0.1654 0.7181 0.0387]
True parameter: [0.02, 1.5, 0.05, 0.18, 0.5, 0.04]
Maximum Absolute Error: 0.5705
Mean Square Error: 0.6112
